# Character-level Language Model
### (CS231n: lecutre 8 - 44)

In [2]:
import torch

In [3]:
vocab = ["h", "e", "l", "o"]
sequence = "hello"


# tokenization(character-level)
seq = list(sequence)                                                   # 문자열을 개별 문자로 분리


# one-hot encoding
encoding_list = []
for i in range(len(seq)):
    initial_encoding = [0] * len(vocab)                                        # 리스트 내부를 모두 0으로 채우기
    for index, value in enumerate(vocab):                              # 인덱스와 인덱스에 해당하는 리스트 요소 가져오기
        if(seq[i] == value):
            initial_encoding[index] = 1
            encoding_list.append(initial_encoding)

encoding = torch.tensor(encoding_list, dtype = torch.float32)       # 리스트를 tensor로 변환

print(encoding)

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])


In [4]:
import numpy as np

In [5]:
# weight matrix 초기화
W_hh = np.random.randn(3, 3)            # 크기가 3x3인 난수 행렬 생성 (정규분포)
W_hh = torch.from_numpy(W_hh).float()   # numpy를 tensor로 변환(dtype을 지정안하면 float64)
print(W_hh.shape)

W_xh = np.random.randn(4, 3)
W_xh = torch.from_numpy(W_xh).float()
print(W_xh.shape)

W_hy = np.random.randn(3, 4)
W_hy = torch.from_numpy(W_hy).float()
print(W_hy.shape)


# initial hidden state 값을 0으로 설정
h_0 = torch.zeros(1, 3)                 # tensor([[0., 0., 0.],
                                              #   [0., 0., 0.],
                                              #   [0., 0., 0.]])
print(h_0.shape)


# weight matrixs와 initial hidden state 값 확인
print("------------------------------------")
print(W_hh)
print(W_xh)
print(W_hy)
print(h_0)

torch.Size([3, 3])
torch.Size([4, 3])
torch.Size([3, 4])
torch.Size([1, 3])
------------------------------------
tensor([[ 0.4641, -0.8993, -1.1127],
        [ 0.3929,  0.0815, -0.2904],
        [-1.4883, -0.4337,  0.2624]])
tensor([[ 2.6494,  0.3771,  0.8982],
        [-0.1352, -0.1540,  1.0252],
        [-0.7746, -1.1512,  1.7684],
        [-0.6189,  0.8291, -0.1519]])
tensor([[ 1.0680, -0.1814,  0.0662, -0.6984],
        [ 0.2446, -0.9202,  0.0109, -1.6046],
        [-0.8233, -0.6297,  0.5547, -2.7709]])
tensor([[0., 0., 0.]])


In [6]:
import torch.nn.functional as F

In [55]:
# <주의>
# * 연산자는 요소별 곱셈을 수행함
# torch.matmul() or @ 연산자가 행렬 곱셈을 수행함


# # hidden state update를 어떻게 구현할지 생각할 용도의 코드
# hidden_state = torch.matmul(h_0, W_hh)
# print(hidden_state)
# print(hidden_state.shape)

# changed_encoding = encoding[0].view(1, 4)
# print(changed_encoding)
# print(changed_encoding.shape)

# print("------------------------------------")

# x = torch.matmul(changed_encoding, W_xh)
# print(x)
# print(x.shape)

# print("------------------------------------")

# updated_hidden_state = hidden_state + x
# print(updated_hidden_state)
# print(updated_hidden_state.shape)

# print("------------------------------------")

# # output
# logits = torch.matmul(updated_hidden_state, W_hy)
# print(logits)
# print(logits.shape)

# y = F.softmax(logits, dim = 1)
# print(y)

# prediction_index = torch.argmax(y)
# prediction = vocab[prediction_index]
# print(prediction)

tensor([[0., 0., 0.]])
torch.Size([1, 3])
tensor([[1., 0., 0., 0.]])
torch.Size([1, 4])
------------------------------------
tensor([[-0.8425, -0.3953, -1.2543]])
torch.Size([1, 3])
------------------------------------
tensor([[-0.8425, -0.3953, -1.2543]])
torch.Size([1, 3])
------------------------------------
tensor([[ 0.1810,  0.9755, -1.3063,  0.4868]])
torch.Size([1, 4])
tensor([[0.2085, 0.4614, 0.0471, 0.2830]])
e


In [12]:
# teacher forcing으로 진행
# (target word를 다음 hidden state의 입력으로 넣어주는 방식)

hidden_state = h_0
prediction_sequence = [seq[0]]

for i in range(len(seq)-1):
    h = torch.matmul(hidden_state, W_hh)

    changed_encoding = encoding[i].view(1, 4)
    x = torch.matmul(changed_encoding, W_xh)

    updated_hidden_state = h + x
    hidden_state = updated_hidden_state                 # hidden state 업데이트

    logits = torch.matmul(updated_hidden_state, W_hy)
    y = F.softmax(logits, dim = 1)                      # 해당 y값이 다음 입력값으로 들어가진 않음
    prediction_index = torch.argmax(y)
    prediction = vocab[prediction_index]
    prediction_sequence.append(prediction)

print(f'target sequence: hello')
print('prediction sequence: ', end = '')
for j in prediction_sequence:
    print(j, end = '')
    

target sequence: hello
prediction sequence: hholo